In [467]:
from bs4 import BeautifulSoup as soup 
import pandas as pd
import pymongo 


In [ ]:
# Initialize Mongo 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.raw_html_db
collection = db.job_detail

In [468]:
# Load data from our job detail collection in our raw_html_db database
data = pd.DataFrame(list(collection.find()))

In [616]:
data.head()

,_id,link_src
0,5e85d38d4284a65cb5c21372,"[<html lang=""en"" class=""artdeco""><head>\n <..."
1,5e85d3964284a65cb5c21373,"[<html lang=""en"" class=""artdeco""><head>\n <..."
2,5e85d39c4284a65cb5c21374,"[<html lang=""en"" class=""artdeco""><head>\n <..."
3,5e85d3a04284a65cb5c21375,"[<html lang=""en"" class=""artdeco""><head>\n <..."
4,5e85d3a54284a65cb5c21376,"[<html lang=""en"" class=""artdeco""><head>\n <..."


In [586]:
# create a list of lists that represent each row in our link src column of our data variable
link_list = []
for i in data.link_src:
    for j in i:
#       convert the raw html into a beautiful soup object we can parse later  
        link_list.extend(soup(j,'html.parser'))


In [617]:
# iterate through each sublist of html, parse it, save the result in a tuple then append it to a list as a dict
headers = []
job_titles = []
for idx in range(len(link_list)):  
    for x in link_list[idx].find_all('h3','jobs-top-card__company-info t-14 mt1'):
        vitals = x.get_text().replace('Company Name','').replace('Company Location','').split('\n')
        vitals = tuple([elem for elem in vitals if elem.strip()])
        headers.append({'company':vitals[0].strip(),'location':vitals[1].strip()})
    try:
        for title in link_list[idx].find('h1',"jobs-top-card__job-title t-24"):
            job_titles.append(title)
    except:
        job_titles.append('n/a')
df1 = pd.DataFrame(headers)
df2 = pd.DataFrame(job_titles)
dataset  = pd.concat([df2,df1],axis=1)
df1.head()

,company,location
0,Gene Therapy Startup,"Pasadena, California"
1,NBS - Local Government Consultants,"Temecula, California"
2,Modis,"Ontario, California, United States"
3,TikTok,"Los Angeles, California, United States"
4,Live Nation Entertainment,"Los Angeles, California, United States"


In [619]:
pd.set_option('display.max_rows', None)
dataset.rename(columns={0:'position'},inplace=True)

In [620]:
dataset.head()

,position,company,location
0,Data Scientist,Gene Therapy Startup,"Pasadena, California"
1,Data Analyst,NBS - Local Government Consultants,"Temecula, California"
2,"Data Analyst with Fortune 10 - Remote Option, ...",Modis,"Ontario, California, United States"
3,Data Analyst,TikTok,"Los Angeles, California, United States"
4,Data Analyst,Live Nation Entertainment,"Los Angeles, California, United States"


In [597]:
job_details = []
for job in range(len(link_list)):
    try:
        for detail in link_list[job].find_all('span','jobs-ppc-criteria__value'):
            job_details.append(detail.get_text())
            print(detail)
    except: 
        print('no data here')
        pass

<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Computer Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Bioinformatics
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              DNA Sequencing
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Pipeline Development
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Bioengineering
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Sequencing
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Machine Learning
            </span>
<span class="jobs-ppc-criter

<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              SQL
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Analytical Skills
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Analysis
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Process Improvement
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Tableau
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Analytics
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Mining
            </span>
<span class="jobs-ppc-criteria__value t-14 t-bla

<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Analytical Skills
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Artificial Intelligence (AI)
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Machine Learning
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Big Data
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Natural Language Processing (NLP)
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Neural Networks
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Team Leadership
            </span>


<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Analytical Skills
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Computer Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data-driven Decision Making
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Engineering
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Product Analytics
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Deep Learning
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Machine Learning
            </span>
<span clas

<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Mathematics
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Analytical Skills
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Solution Implementation
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Engineering
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Deep Learning
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Machine Learning
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Problem Solving
            </span>
<span class="jobs-ppc

<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Mathematics
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Computer Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Data Science
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Next Gen
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Image Processing
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Apache Spark
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Deep Learning
            </span>
<span class="jobs-ppc-criteria__value t-14 t-black t-normal ml2 block">
              Machine Learning
            </span>
<span class="jobs-ppc-criteria__value t-

In [609]:
final_job_skills= pd.DataFrame(job_details)
final_job_skills= final_job_details.apply(lambda x: x.str.strip())
final_job_skills = final_job_skills.rename(columns={0:'skills'})

In [614]:
final_job_skills.to_csv('../data/updated_job_skills.csv')

In [615]:
dataset.to_csv('../data/jobs_listing.csv')

In [480]:
db = client.raw_html_db
collection_1 = db.html

In [498]:
raw = pd.DataFrame(list(collection_1.find()))

In [527]:
html_soup = []

for i in raw.job:
    for j in i:
        html_soup.extend(soup(j,'html.parser'))
   

In [529]:
titles = []
for x in range(len(html_soup)):
    for a in html_soup[x].find_all('a','job-card-search__link-wrapper js-focusable disabled ember-view'):
            titles.append(a.text.strip())

In [537]:
# employers = []
# for x in range(len(html_soup)):
#     for a in html_soup[x].find_all('a','job-card-search__company-name-link t-normal ember-view', href=True):
#         employers.append(a.text.strip())

        

In [536]:
# employers


In [506]:
# locations = []
# for x in range(len(html_soup)):
#     for a in html_soup[x].find_all('span',"job-card-search__location artdeco-entity-lockup__caption ember-view"):
#         locations.append(a.text.strip())